In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import paho.mqtt.client as mqtt
import time
# Define the MQTT broker details
broker = "mqtt.119.ovh"
port = 1883
topic = "diya06"
read_topic = topic + "/rx/"
transmit_topic = topic + "/tx/"
keep_alive = 60

last_message = None

# Callback function when a message is received
def on_message(client, userdata, message):
    last_message = message.payload.decode()
    print(f"Message received: {last_message} on topic {message.topic}")

    print(type(last_message))
    parts = last_message.split()
    if len(parts) < 9:
            print("Received incomplete message")
            return
    print(type(parts))
    print(parts)

    # Parse each component based on its position in the message
    timestamp = parts[0]
    sensor_label = parts[1]
    T_1 = float(parts[2])
    T_2 = float(parts[3])
    T_3 = float(parts[4])

    # Parse the 'Peltier' and 'Fan' values by removing their labels
    peltier_value = int(parts[6])
    fan_value = int(parts[8])

    # Print out parsed values
    print("Timestamp:", timestamp)
    print("Measurements:", T_1, T_2, T_3)
    print("Peltier Value:", peltier_value)
    print("Fan Value:", fan_value)

def send_control_input(client:mqtt.Client, x_HP:int, x_FAN:int):
    sign_HP = "+" if x_HP >= 0 else "-"

    # Bound control values
    x_HP = max(min(x_HP, 100), -100)
    x_FAN = max(min(x_FAN, 100), 0)

    # Format control values
    x_HP_formatted = f"{sign_HP}{abs(int(x_HP)):03d}"
    x_FAN_formatted = f"{int(x_FAN):03d}"

    message_str = f"S{x_HP_formatted}{x_FAN_formatted}"

    client.publish(transmit_topic, message_str)
    print(f"Message sent: x_HP = {x_HP}, x_FAN = {x_FAN}")


# Create an MQTT client instance
# client = mqtt.Client()
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# Assign the on_message callback
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, keep_alive)

# Subscribe to the specified topic
client.subscribe(read_topic)

In [75]:
x_HP = 0
x_FAN = 0

send_control_input(client, x_HP, x_FAN)

try:
    client.loop_start()

except KeyboardInterrupt:
    client.loop_stop()
    client.disconnect()

Message sent: x_HP = 0, x_FAN = 0


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

Message received: 20000102T161345	TMP&U	 26.26	 26.85	 34.63	Peltier: 100	Fan: 0	 on topic diya06/rx/
<class 'str'>
<class 'list'>
['20000102T161345', 'TMP&U', '26.26', '26.85', '34.63', 'Peltier:', '100', 'Fan:', '0']
Timestamp: 20000102T161345
Measurements: 26.26 26.85 34.63
Peltier Value: 100
Fan Value: 0
Message received: 20000102T161346	TMP&U	 26.27	 26.85	 34.68	Peltier: 100	Fan: 0	 on topic diya06/rx/
<class 'str'>
<class 'list'>
['20000102T161346', 'TMP&U', '26.27', '26.85', '34.68', 'Peltier:', '100', 'Fan:', '0']
Timestamp: 20000102T161346
Measurements: 26.27 26.85 34.68
Peltier Value: 100
Fan Value: 0


Message received: State:	1 on topic diya06/rx/
<class 'str'>
Received incomplete message
Message received: rxBuffer:	S+000000 on topic diya06/rx/
<class 'str'>
Received incomplete message
Message received: Setting thermal actuators on topic diya06/rx/
<class 'str'>
Received incomplete message
Message received: Read 0	0 on topic diya06/rx/
<class 'str'>
Received incomplete message
Message received: Set	Peltier=0	Fan=0 on topic diya06/rx/
<class 'str'>
Received incomplete message
Message received: 20000102T161347	TMP&U	 26.28	 26.85	 34.74	Peltier: 0	Fan: 0	 on topic diya06/rx/
<class 'str'>
<class 'list'>
['20000102T161347', 'TMP&U', '26.28', '26.85', '34.74', 'Peltier:', '0', 'Fan:', '0']
Timestamp: 20000102T161347
Measurements: 26.28 26.85 34.74
Peltier Value: 0
Fan Value: 0
Message received: State:	-1 on topic diya06/rx/
<class 'str'>
Received incomplete message
Message received: rxBuffer:	S+000000 on topic diya06/rx/
<class 'str'>
Received incomplete message
Message received: 200001

In [77]:
client.loop_stop()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [31]:
client.disconnect()

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>